# MIMIC 4 data - dataset construction labevents

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [4]:
adm = pd.read_csv("/path/admissions_processed.csv")
adm.tail()

,Unnamed: 0,subject_id,anchor_age,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,elapsed_time,elapsed_days,deathtag
16567,524311,19959116,47,25551829,2128-11-06 10:25:00,2128-11-14 17:30:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,?,MARRIED,HISPANIC/LATINO,2128-11-06 07:39:00,2128-11-06 11:51:00,0,8 days 07:05:00,8,0
16568,524320,19962250,58,23717024,2125-12-03 02:27:00,2125-12-11 12:58:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2125-12-03 01:11:00,2125-12-03 04:07:00,0,8 days 10:31:00,8,0
16569,524446,19984270,72,20925564,2185-10-29 18:24:00,2185-11-07 15:56:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,8 days 21:32:00,8,0
16570,524486,19992202,80,20329411,2153-02-26 00:48:00,2153-03-02 17:00:00,NaN,URGENT,TRANSFER FROM SKILLED NURSING FACILITY,SKILLED NURSING FACILITY,Medicare,ENGLISH,DIVORCED,WHITE,2153-02-25 18:11:00,2153-02-26 01:58:00,0,4 days 16:12:00,4,0
16571,524518,19999068,63,21606769,2161-08-24 04:10:00,2161-09-02 19:00:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,DIVORCED,WHITE,2161-08-24 03:14:00,2161-08-24 05:26:00,0,9 days 14:50:00,9,0


In [11]:
lab = pd.read_csv("/path/labevents.csv.gz")

In [ ]:
#Restrict the dataset to the previously selected admission ids only.
adm_ids=list(adm["hadm_id"])
lab=lab.loc[lab["hadm_id"].isin(adm_ids)]
print("Number of patients remaining in the database: ")
print(lab["subject_id"].nunique())

In [ ]:
#item_id
item_id=pd.read_csv("/path/d_labitems.csv.gz")
item_id_1=item_id[["itemid","label"]]
item_id_1.head()

intermediate save due to memory issues

In [ ]:
lab3.to_csv("/path/lab_processed(intermediate).csv")
lab3 = pd.read_csv("/path/lab_processed(intermediate).csv")

In [ ]:
#We merge the name of the item administrated.
lab2=pd.merge(lab3,item_id_1,on="itemid")
lab2.head()
print("Number of patients remaining in the database: ")
print(lab2["subject_id"].nunique())

In [ ]:
n_best=150
#For each item, evaluate the number of patients who have been given this item.
pat_for_item=lab2.groupby("label")["subject_id"].nunique()
#Order by occurence and take the 20 best (the ones with the most patients)
frequent_labels=pat_for_item.sort_values(ascending=False)[:n_best]

#Select only the time series with high occurence.
lab3=lab2.loc[lab2["label"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(lab3["subject_id"].nunique())

In [ ]:
#Only select the subset that was used in the paper (only missing is INR(PT))
subset=["Albumin","Alanine Aminotransferase (ALT)","Alkaline Phosphatase","Anion Gap","Asparate Aminotransferase (AST)","Base Excess","Basophils","Bicarbonate","Bilirubin, Total","Calcium, Total","Calculated Total CO2","Chloride","Creatinine","Eosinophils","Glucose","Hematocrit","Hemoglobin",
"Lactate","Lymphocytes","MCH","MCV","Magnesium","Monocytes","Neutrophils","PT","PTT","Phosphate","Platelet Count","Potassium","RDW","Red Blood Cells","Sodium","Specific Gravity","Urea Nitrogen","White Blood Cells","pCO2","pH","pO2"]

lab3=lab3.loc[lab3["label"].isin(subset)].copy()

In [ ]:
print(lab3.groupby("label")["valueuom"].value_counts())

In [ ]:
lab3.to_csv("/path/lab_processed.csv")